## Exercise 1: Explore filter parameters

In this exercises you will explore how different parameters affect the filtering process.

1. Import the data from `data/moving_lfp.pickle` (I should already have this)
2. Take a few seconds of the data and filter them in different frequency bands, plotting the resulting signal. Fix the central frequency and play with the width of the band. The, try to change the central frequency of the band.
3. Change the order of the filter whilst keeping the frequency band fixed. How does the signal change? How does the computing time change? 